In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import copy

In [ ]:
from utils.ga import GenomeBinary, GenomeFloat, GA
from utils.bp import train_model, reset_model

# Generate Data

In [ ]:
X_BOUND = [-10, 10]
def F(x):
    # return x + 10*np.sin(5*x) + 7*np.cos(4*x)
    return 1 / (1 + np.exp(x))

In [ ]:
data_size = 1000
noise = 0.1
x_origin = (X_BOUND[0] + (X_BOUND[1]-X_BOUND[0]) * np.random.rand(data_size))
y_origin = F(x_origin) + noise * np.random.rand(data_size)

# Build Model

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Working on {}'.format(device))
x_tensor = torch.from_numpy(x_origin).unsqueeze_(-1).float().to(device)
y_tensor = torch.from_numpy(y_origin).unsqueeze_(-1).float().to(device)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(1, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 1),
).to(device)

In [ ]:
def params_to_chrom(params):
    chrom = np.empty(0)
    for key in params:
        chrom = np.append(chrom, params[key].cpu().numpy().flatten(), axis=-1)
    return chrom

def chrom_to_params(chrom, params_template):
    params = copy.deepcopy(params_template)
    idx = 0
    for key in params:
        param_length = np.prod(params_template[key].shape)
        param = torch.from_numpy(chrom[idx: idx+param_length].reshape(params_template[key].shape)).to(device)
        params[key] = param
        idx += param_length
    return params

In [ ]:
params_template = copy.deepcopy(model.state_dict())
chrom_len = 0
bound_l = np.empty(0)
bound_h = np.empty(0)
for key in params_template:
    param_length = np.prod(params_template[key].shape)
    if 'weight' in key:
        # kaiming uniform
        weight = params_template[key]
        fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(weight)
        gain = torch.nn.init.calculate_gain('relu')
        _bound = gain * np.sqrt(3 / fan_in)
    elif 'bias' in key:
        fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(weight)
        _bound = 1 / np.sqrt(fan_in)
    else:
        raise Exception('Unknow parameter')
    bound_l = np.append(bound_l, -np.ones(param_length)*_bound)
    bound_h = np.append(bound_h, np.ones(param_length)*_bound)
    chrom_len += param_length
bound = np.array([bound_l, bound_h])

In [ ]:
def calculate_fitness(chrom):
    params = chrom_to_params(chrom, params_template)
    model.load_state_dict(params)
    loss = train_model(model, x_tensor, y_tensor, num_epoches=100, batch_size=8192, learning_rate=1e-3, log=False)
    fitness = 1./loss
    return fitness

In [ ]:
pop_size = 100
num_gen = 1000
ga = GA(pop_size, chrom_len, bound, calculate_fitness, GenomeClass=GenomeFloat, cross_prob=0.8, mutate_prob=0.3)
ga.genetic(num_gen, log=True)

In [ ]:
# best_ga_params = chrom_to_params(ga.genome.view_best(ga.bound), params_template)
best_ga_params = chrom_to_params(ga.genome.view(0, ga.bound), params_template)
model.load_state_dict(best_ga_params)
train_model(model, x_tensor, y_tensor, num_epoches=100, batch_size=8192, learning_rate=1e-3, log=True)

x_axis = np.linspace(*X_BOUND, 200)
plt.plot(x_axis, F(x_axis))
plt.scatter(x_origin, np.squeeze(model(x_tensor).cpu().detach().numpy(), -1), color='r')
plt.show()